# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [112]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

In [113]:
A = sm.symbols('A')
X = sm.symbols('X')
Alpha = sm.symbols('alpha')
c = sm.symbols('c')
n = sm.symbols('n_t')
I = sm.symbols('I')
Y = sm.symbols('Y')
L = sm.symbols('L_t')
beta = sm.symbols('beta')
Lambda = sm.symbols('lambda')
p = sm.symbols('p')
b = sm.symbols('b')
sigma = sm.symbols('sigma')
mu = sm.symbols('mu')

In [114]:
objective = sm.ln(c)+ beta * sm.ln(n)
objective

beta*log(n_t) + log(c)

In [115]:
budget_constraint = sm.Eq(c+Lambda * n + p * b,I)
budget_constraint

Eq(b*p + c + lambda*n_t, I)

In [116]:
size = sm.Eq(sigma * b,n)
size

Eq(b*sigma, n_t)

In [117]:
c_from_con = sm.solve(budget_constraint, c)
c_from_con[0]

I - b*p - lambda*n_t

In [118]:
b_size = sm.solve(size, b)
b_size[0]

n_t/sigma

In [119]:
objective_subs = objective.subs(c, c_from_con[0])
objective_subs

beta*log(n_t) + log(I - b*p - lambda*n_t)

In [120]:
objective_subs_1 = objective_subs.subs(b, b_size[0])
objective_subs_1

beta*log(n_t) + log(I - lambda*n_t - n_t*p/sigma)

In [121]:
foc = sm.diff(objective_subs_1, n)
foc

beta/n_t + (-lambda - p/sigma)/(I - lambda*n_t - n_t*p/sigma)

In [122]:
sol = sm.solve(sm.Eq(foc,0), n)
sol[0]

I*beta*sigma/(beta*lambda*sigma + beta*p + lambda*sigma + p)

In [123]:
Income = sm.Eq(A*(X/L)**(1-Alpha),I)
Income

Eq(A*(X/L_t)**(1 - alpha), I)

In [124]:
Population = sm.Eq(n*L+(1-mu)*L,L)
Population

Eq(L_t*n_t + L_t*(1 - mu), L_t)

In [125]:
Population_1 = Population.subs(n, sol[0])
Population_1

Eq(I*L_t*beta*sigma/(beta*lambda*sigma + beta*p + lambda*sigma + p) + L_t*(1 - mu), L_t)

In [126]:
Population_2 = Population_1.subs(I, Income.lhs)
Population_2

Eq(A*L_t*beta*sigma*(X/L_t)**(1 - alpha)/(beta*lambda*sigma + beta*p + lambda*sigma + p) + L_t*(1 - mu), L_t)

In [130]:
sol_1 = sm.solve(sm.Eq(Population_2.lhs,0), L)
sol_1[0]

X**(alpha/(alpha - 1))/(A*X*beta*sigma/(beta*lambda*mu*sigma - beta*lambda*sigma + beta*mu*p - beta*p + lambda*mu*sigma - lambda*sigma + mu*p - p))**(1/(alpha - 1))

In [131]:
Income_1 = Income.subs(L, sol_1[0])
Income_1

Eq(A*(X*(A*X*beta*sigma/(beta*lambda*mu*sigma - beta*lambda*sigma + beta*mu*p - beta*p + lambda*mu*sigma - lambda*sigma + mu*p - p))**(1/(alpha - 1))/X**(alpha/(alpha - 1)))**(1 - alpha), I)

In [138]:
beta = 0.5 
lamda = 0.3 
p =  0.3 
sigma = 0.2 
A = 1
X =1 
mu = 0.3 
alpha = 0.3 
obj = lambda Lss: Lss - beta/((1+beta)*(lamda+p/sigma))*A*X**(1-alpha)*Lss**alpha+(1-mu)*Lss
result = optimize.root_scalar(obj,bracket=[0.1,100],method='brentq')

print('the steady state for L is',result.root)

ValueError: f(a) and f(b) must have different signs

In [137]:
beta = 0.2
lambda = 0.02
p = 0.01
alpha = 1/3
sigma = 0.1
my =0.5

# write your code here

f = lambda L: L**alpha
obj_Lss = lambda Lss: Lss - (A*X**(1-alpha)*(Lss) + (1-my)*Lss)/((1+beta)*(lambda+(p/sigma)))
result = optimize.root_scalar(obj_Lss,bracket=[0.1,100],method='brentq')

print('the steady state for L is',result.root)

ValueError: f(a) and f(b) must have different signs

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 